In [1]:
import sys, os
import argparse

import numpy as np
import pandas as pd
import uproot
import csv, yaml

import h5py
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

In [2]:
runnum = os.listdir('/store/hep/users/yewzzang/JSNS2/com_data')

In [3]:
path = '/store/hep/users/yewzzang/JSNS2/com_data/'
FN = '/FN_cut_Rho_1.6_ZL_1.0'
ME = '/ME_cut_Rho_1.6_ZL_1.0'

In [4]:
exist = []
nofile = []
runs = []
for i in range(len(runnum)):
    
    if os.path.exists(path+runnum[i]+FN):
        exist.append(len(os.listdir(path+runnum[i]+FN)))
        runs.append(runnum[i][3:])
    else:
        nofile.append(runnum[i])

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/users/yewzzang/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-f31057d4219e>", line 7, in <module>
    exist.append(len(os.listdir(path+runnum[i]+FN)))
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/users/yewzzang/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/users/yewzzang/.local/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1148, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/users/yewzzang/

KeyboardInterrupt: 

In [ ]:
runss =[ '1596','1597', '1598', '1599', '1600', '1606', '1608', '1609', '1614', '1621', '1623', '1624', '1625', '1626', '1627', '1628', '1629',  '1630', '1631', '1632', '1633', '1634', '1635', '1636', '1637',    '1638', '1639', '1640', '1641', '1642', '1644', '1645', '1646',  '1647', '1649', '1650', '1651', '1652', '1653', '1654', '1655', '1656', '1658', '1659', '1660', '1661', '1662', '1663', '1664',  '1665', '1666', '1667', '1668', '1669', '1671', '1672', '1673',   '1674', '1675', '1676', '1677', '1678', '1679', '1680', '1681',  '1682', '1683', '1684', '1685', '1686', '1687', '1689', '1692','1693', '1711', '1716', '1717', '1718', '1719', '1722', '1724','1725', '1726', '1727', '1728', '1729', '1732', '1749', '1750','1759', '1760', '1761', '1762', '1763', '1766', '1768', '1769','1770', '1784', '1785', '1787', '1789', '1790', '1795', '1796','1797', '1798', '1799', '1807', '1809', '1810', '1811', '1812','1813', '1814', '1815', '1817', '1818', '1819', '1821',  '1826', '1828', '1829', '1830', '1831', '1833', '1835','1836', '1837', '1838', '1839', '1843', '1849', '1850', '1851', '1852', '1853', '1854', '1864', '1865', '1867', '1868','1870', '1872', '1873', '1874', '1875', '1876', '1877', '1878','1879', '1880', '1891', '1909', '1910', '1911', '1912', '1913','1914', '1915', '1916', '1917', '1918', '1919', '1920', '1921','1922', '1923', '1924', '1925', '1926', '1928', '1930', '1932', '1933', '1934', '1935', '1936', '1937', '1939', '1940', '1941', '1942', '1943', '1944', '1945', '1948', '1949', '1950', '1952']

In [ ]:
run_num = []
FN_num = []
ME_num = []
FN_rej = []
ME_eff = []
AUC = []

for j in runss:

    if os.path.exists('result/eval_'+str(j)+'_model_r01592_R14_Z10_nolog/eval_'+str(j)+'_model_r01592_R14_Z10_nolog_information.txt') == True:
        f = open('result/eval_'+str(j)+'_model_r01592_R14_Z10_nolog/eval_'+str(j)+'_model_r01592_R14_Z10_nolog_information.txt', 'r')
        lines = f.readlines()
    #     for line in lines:
    #         print(line)
        run_num.append(j)
#         print(lines)
#         print(float(lines[-2].split(' ')[-2]))
#         print(float(lines[-2].split(' ')[-1][:-1]))
#         print(j)
        FN_num.append(float(lines[-2].split(' ')[-2]))        #### FN num
        ME_num.append(float(lines[-2].split(' ')[-1][:-1]))   #### ME num
        ME_eff.append(float(lines[0].split(' ')[-1][:-2])*100)  #### ME effi
        FN_rej.append((1-float(lines[1].split(' ')[-1][:-2]))*100)    #### FN rej
        AUC.append(float(lines[-1][:-2]))                       #### AUC
        f.close()
    

In [ ]:
a = 0
c = 0
sets = []
setss = []
for i in range(len(np.array(FN_num))):
    
    b = np.array(FN_num)[i]
    a = a + b
    sets.append(np.array(runss)[i])
    if a > 7000:
        print(a,i, np.array(runss)[i])
        c = c+1
        a = 0
        print(c,'c')
        setss.append(sets)
        sets = []

In [ ]:


for k in range(len(setss)):
    
    infos = []
    sis = []

    preds = []
    labels = []
    dVertexs = []
    dTs = []
    vertexXs = []
    vertexYs = []
    vertexZs = []

    result_output = 'eval_'+str(k+1).zfill(4)

    for i in setss[k]:
        result_input = 'eval_'+str(i) +'_model_r01592_R14_Z10_nolog'
        print(result_input)

        si_path = 'result/' + result_input + '/sampleInfo_eval.csv'
        fPred = 'result/' + result_input + '/' + result_input + '.csv'
        si = pd.read_csv(si_path)
        info = pd.read_csv(fPred)

        info_numpy = np.array(info)
        si_numpy = np.array(si)


        for i in range(len(info_numpy)):

            label = info_numpy[i][0]
            pred = info_numpy[i][1]

            fileidx = info_numpy[i][4]

            filename = si_numpy[int(fileidx)][2]


            idx = info_numpy[i][5]


            data = h5py.File(filename,'r')

            dT = data['events']['dT'][idx]
            dVertex = data['events']['dVertex'][idx]
            vertexX = data['events']['vertexX'][idx]
            vertexY = data['events']['vertexY'][idx]
            vertexZ = data['events']['vertexZ'][idx]


            preds.append(pred)
            labels.append(label)
            dVertexs.append(dVertex)
            dTs.append(dT)
        #     print(dTs.type)
            vertexXs.append(vertexX)
            vertexYs.append(vertexY)
            vertexZs.append(vertexZ)





    preds = np.array(preds)
    labels = np.array(labels)
    dVertexs = np.array(dVertexs)
    dTs = np.array(dTs)
    vertexXs = np.array(vertexXs)
    vertexYs = np.array(vertexYs)
    vertexZs = np.array(vertexZs)


    ME_label = []
    ME_dVertex = []
    ME_dT = []
    ME_vertexX = []
    ME_vertexY = []
    ME_vertexZ = []
    ME_pred = []


    FN_label = []
    FN_dVertex = []
    FN_dT = []
    FN_vertexX = []
    FN_vertexY = []
    FN_vertexZ = []
    FN_pred = []

    for i in range(len(preds)):
        if labels[i] == 1:
            ME_label.append(labels[i])
            ME_dVertex.append(dVertexs[i])
            ME_dT.append(dTs[i])
            ME_vertexX.append(vertexXs[i])
            ME_vertexY.append(vertexYs[i])
            ME_vertexZ.append(vertexZs[i])
            ME_pred.append(preds[i])
        else:

            FN_label.append(labels[i])
            FN_dVertex.append(dVertexs[i])
            FN_dT.append(dTs[i])
            FN_vertexX.append(vertexXs[i])
            FN_vertexY.append(vertexYs[i])
            FN_vertexZ.append(vertexZs[i])
            FN_pred.append(preds[i])

    ME_sig = -np.log((1/np.array(ME_pred))-1)
    FN_sig = -np.log((1/np.array(FN_pred))-1)
    FN_over_0 = 100*np.sum(FN_sig > 0)/len(FN_sig)
    ME_under_0 = 100*np.sum(ME_sig < 0)/len(ME_sig)


    ##################plot CNN score distribution figure
    plt.hist(FN_sig, bins = 100, range = (-20, 20), density = True, color ='r',histtype = 'step')
    plt.hist(ME_sig, bins = 100, range = (-20, 20), density = True, color ='b',histtype = 'step')
    # plt.text(10, 0.15,'FN > 0' + str(FN_over_0), fontsize = 20)
    # plt.text(10, 0.1, 'ME < 0' + str(ME_under_0), fontsize = 20)


    FN_over = '%.2f' %FN_over_0
    ME_under = '%.2f' %ME_under_0

    label = ['FN : FN > 0 = '+FN_over+'%', 
             'ME : ME < 0 = '+ME_under+'%']


    leg = plt.legend(label, loc = 'best', frameon=False)

    leg_lines = leg.get_lines()
    leg_texts = leg.get_texts()

    plt.setp(leg_lines, linewidth=15)
    plt.setp(leg_texts, fontsize=15)

    plt.title('')
    plt.savefig('result/' + result_output + '/' + result_output + '_CNN score distribution.png', dpi=300)
    plt.clf()



    num_FN = len(FN_dT)
    num_ME = len(ME_dT)
    ###########plot dT
    plt.hist(np.array(FN_dT)*0.001, bins = 100, color= 'r', alpha = 0.5, density = True, histtype = 'step')
    plt.hist(np.array(ME_dT)*0.001, bins = 100, color= 'b', alpha = 0.5, density = True, histtype = 'step')

    plt.xticks(fontsize = 15)
    plt.yticks(fontsize = 15)
    plt.xlabel("\u03BCs", fontsize=15, loc='right')
    plt.savefig('result/' + result_output + '/' + result_output + '_dT.png', dpi=300)
    plt.clf()

    ###########plot dVertex
    plt.hist(np.array(FN_dVertex)*100, bins = 80, color= 'r', alpha = 0.5, density = True, histtype = 'step')
    plt.hist(np.array(ME_dVertex)*100, bins = 80, color= 'b', alpha = 0.5, density = True, histtype = 'step')

    plt.xticks(fontsize = 15)
    plt.yticks(fontsize = 15)
    plt.xlabel("cm", fontsize=15, loc='right')
    plt.savefig('result/' + result_output + '/' + result_output + '_dVertex.png', dpi=300)
    plt.clf()


    ##############plot vertexZ
    plt.hist(np.array(FN_vertexZ)*100, bins = 80, color= 'r', alpha = 0.5, density = True, histtype = 'step')
    plt.hist(np.array(ME_vertexZ)*100, bins = 80, color= 'b', alpha = 0.5, density = True, histtype = 'step')

    plt.xticks(fontsize = 15)
    plt.yticks(fontsize = 15)
    plt.xlabel("cm", fontsize=15, loc='right')
    plt.savefig('result/' + result_output + '/' + result_output + '_vertexZ.png', dpi=300)
    plt.clf()


    ##############plot R2
    plt.hist((np.array(FN_vertexX)**2+np.array(FN_vertexY)**2)*100, bins = 80, color= 'r', alpha = 0.5, density = True, histtype = 'step')
    plt.hist((np.array(ME_vertexX)**2+np.array(ME_vertexY)**2)*100, bins = 80, color= 'b', alpha = 0.5, density = True, histtype = 'step')

    plt.xticks(fontsize = 15)
    plt.yticks(fontsize = 15)
    plt.xlabel("cm", fontsize=15, loc='right')
    plt.savefig('result/' + result_output + '/' + result_output + '_vertex_R2.png', dpi=300)
    plt.clf()

    ####################################################################################

    list_range = np.arange(0,1,0.0001)


    for i in range(len(list_range)):
        a = len(np.array(ME_pred)[np.array(ME_pred)>list_range[i]])/len(np.array(ME_pred))
        b = len(np.array(FN_pred)[np.array(FN_pred)<list_range[i]])/len(np.array(FN_pred))
    #     print(list_range[i])

        if (a > 0.99):

            eff_99 = list_range[i]
         

        if (a > 0.95):

            eff_95 = list_range[i]
  

        if (a > 0.90):

            eff_90 = list_range[i]
         
            
#     
        if ((0.903 > b)&(b > 0.897)):

            eff_90_range = list_range[i]
       

        if ((0.953 > b) & (b > 0.947)):
#         if (b < 0.95):

            eff_95_range = list_range[i]
   

        if ((0.993 > b) & (b > 0.987)):
#         if (b < 0.99):
            eff_99_range = list_range[i]
         
    ##### eff 0.5 pred
    eff_50_ME = len(np.array(ME_pred)[np.array(ME_pred)>0.5])/len(np.array(ME_pred))
    eff_50_FN = len(np.array(FN_pred)[np.array(FN_pred)>0.5])/len(np.array(FN_pred))
    num_50_ME = num_ME*len(np.array(ME_pred)[np.array(ME_pred)>0.5])/len(np.array(ME_pred))
    num_50_FN = num_FN*len(np.array(FN_pred)[np.array(FN_pred)>0.5])/len(np.array(FN_pred))



    ##### eff 99% efficient
    eff_99_ME = len(np.array(ME_pred)[np.array(ME_pred)>eff_99])/len(np.array(ME_pred))
    eff_99_FN = len(np.array(FN_pred)[np.array(FN_pred)>eff_99])/len(np.array(FN_pred))
    num_99_ME = num_ME*len(np.array(ME_pred)[np.array(ME_pred)>eff_99])/len(np.array(ME_pred))
    num_99_FN = num_FN*len(np.array(FN_pred)[np.array(FN_pred)>eff_99])/len(np.array(FN_pred))




    ##### eff 95% efficient
    eff_95_ME = len(np.array(ME_pred)[np.array(ME_pred)>eff_95])/len(np.array(ME_pred))
    eff_95_FN = len(np.array(FN_pred)[np.array(FN_pred)>eff_95])/len(np.array(FN_pred))
    num_95_ME = num_ME*len(np.array(ME_pred)[np.array(ME_pred)>eff_95])/len(np.array(ME_pred))
    num_95_FN = num_FN*len(np.array(FN_pred)[np.array(FN_pred)>eff_95])/len(np.array(FN_pred))


    ##### eff 90% efficient
    eff_90_ME = len(np.array(ME_pred)[np.array(ME_pred)>eff_90])/len(np.array(ME_pred))
    eff_90_FN = len(np.array(FN_pred)[np.array(FN_pred)>eff_90])/len(np.array(FN_pred))
    num_90_ME = num_ME*len(np.array(ME_pred)[np.array(ME_pred)>eff_90])/len(np.array(ME_pred))
    num_90_FN = num_FN*len(np.array(FN_pred)[np.array(FN_pred)>eff_90])/len(np.array(FN_pred))
    
    
    
    
    ##### eff 99%_FN efficient
    eff_99_ME_FN = len(np.array(ME_pred)[np.array(ME_pred)<eff_99_range])/len(np.array(ME_pred))
    eff_99_FN_FN = len(np.array(FN_pred)[np.array(FN_pred)<eff_99_range])/len(np.array(FN_pred))
    num_99_ME_FN = num_ME*len(np.array(ME_pred)[np.array(ME_pred)<eff_99_range])/len(np.array(ME_pred))
    num_99_FN_FN = num_FN*len(np.array(FN_pred)[np.array(FN_pred)<eff_99_range])/len(np.array(FN_pred))


    ##### eff 95%_FN efficient
    eff_95_ME_FN = len(np.array(ME_pred)[np.array(ME_pred)<eff_95_range])/len(np.array(ME_pred))
    eff_95_FN_FN = len(np.array(FN_pred)[np.array(FN_pred)<eff_95_range])/len(np.array(FN_pred))
    num_95_ME_FN = num_ME*len(np.array(ME_pred)[np.array(ME_pred)<eff_95_range])/len(np.array(ME_pred))
    num_95_FN_FN = num_FN*len(np.array(FN_pred)[np.array(FN_pred)<eff_95_range])/len(np.array(FN_pred))


    ##### eff 90%_FN efficient
    eff_90_ME_FN = len(np.array(ME_pred)[np.array(ME_pred)<eff_90_range])/len(np.array(ME_pred))
    eff_90_FN_FN = len(np.array(FN_pred)[np.array(FN_pred)<eff_90_range])/len(np.array(FN_pred))
    num_90_ME_FN = num_ME*len(np.array(ME_pred)[np.array(ME_pred)<eff_90_range])/len(np.array(ME_pred))
    num_90_FN_FN = num_FN*len(np.array(FN_pred)[np.array(FN_pred)<eff_90_range])/len(np.array(FN_pred))


    f = open('result/' + result_output + '/' + result_output + '_efficiency.txt','w')
    print('         |   90%   |   95%   |   99%   |   mid   |',file=f)
    print('--------------------------------------------------',file=f)
    print('  ME_eff |','%.4f  |'%eff_90_ME,'%.4f  |'%eff_95_ME,'%.4f  |'%eff_99_ME,'%.4f  |'%eff_50_ME,file=f)
    print('--------------------------------------------------',file=f)
    print('  FN_eff |','%.4f  |'%eff_90_FN,'%.4f  |'%eff_95_FN,'%.4f  |'%eff_99_FN,'%.4f  |'%eff_50_FN,file=f)
    print('--------------------------------------------------',file=f)
    print('ME_remain|','%7d'%int(num_90_ME),'|','%7d'%int(num_95_ME),'|','%7d'%int(num_99_ME),'|','%7d'%int(num_50_ME),'|',file=f)
    print('--------------------------------------------------',file=f)
    print('FN_remain|','%7d'%int(num_90_FN),'|','%7d'%int(num_95_FN),'|','%7d'%int(num_99_FN),'|','%7d'%int(num_50_FN),'|',file=f)
    print('--------------------------------------------------',file=f)
    print('  ME_eff |','%.4f  |'%eff_90_ME_FN,'%.4f  |'%eff_95_ME_FN,'%.4f  |'%eff_99_ME_FN,'%.4f  |'%eff_50_ME,file=f)
    print('--------------------------------------------------',file=f)
    print('  FN_eff |','%.4f  |'%eff_90_FN_FN,'%.4f  |'%eff_95_FN_FN,'%.4f  |'%eff_99_FN_FN,'%.4f  |'%eff_50_FN,file=f)
    print('--------------------------------------------------',file=f)
    print('ME_remain|','%7d'%int(num_90_ME_FN),'|','%7d'%int(num_95_ME_FN),'|','%7d'%int(num_99_ME_FN),'|','%7d'%int(num_50_ME),'|',file=f)
    print('--------------------------------------------------',file=f)
    print('FN_remain|','%7d'%int(num_90_FN_FN),'|','%7d'%int(num_95_FN_FN),'|','%7d'%int(num_99_FN_FN),'|','%7d'%int(num_50_FN),'|',file=f)
    print('--------------------------------------------------',file=f)
    print('CNN score|','','%.3f'%eff_90,' | ','%.3f'%eff_95,' | ','%.3f'%eff_99,' |  ',0.5,'  |',file=f)
    print('--------------------------------------------------',file=f)
    print('CNN score|','','%.3f'%eff_90_range,' | ','%.3f'%eff_95_range,' | ','%.3f'%eff_99_range,' |  ',0.5,'  |',file=f)
    print('==================================================',file=f)
    print('         |   FN    |   ME    |',file=f)
    print('# data   |','%7d'%len(FN_dT),'|', '%7d'%len(ME_dT),'|',file=f)

    f.close()



    ###############plot evaluation 

    ##### Draw ROC curve #####
    from sklearn.metrics import roc_curve, roc_auc_score
    from matplotlib.cbook import get_sample_data

    df = pd.read_csv(fPred)
    tpr, fpr, thr = roc_curve(df['label'], df['prediction'], sample_weight=df['weight'], pos_label=0)
    auc = roc_auc_score(df['label'], df['prediction'], sample_weight=df['weight'])


    df_bkg = df[df.label==0]
    df_sig = df[df.label==1]
    plt.rcParams['figure.figsize'] = (10, 10)

    plt.hist(df_bkg['prediction']*100, weights=df_bkg['weight'], histtype='step', 
             density=True, bins=50, color='red', linewidth=3)

    plt.hist(df_sig['prediction']*100, weights=df_sig['weight'], histtype='step', 
             density=True, bins=50, color='blue', linewidth=3)



    plt.xticks(np.arange(0, 101, step=20),["{}".format(x*0.01) for x in np.arange(0, 101,step=20)],fontsize = 10)
    plt.yticks(fontsize = 10)
    plt.xlabel("CNN score", fontsize=15, loc='right')
    plt.ylabel("Normalized", fontsize=15, loc='top')
    plt.xlim(0, 100)
    label = ['Fast Neutron', 'Michel Electrons']

    leg = plt.legend(label, loc = 'upper center', frameon=False)

    leg_lines = leg.get_lines()
    leg_texts = leg.get_texts()

    plt.setp(leg_lines, linewidth=10)
    plt.setp(leg_texts, fontsize=10)

    plt.savefig('result/' + result_output + '/' + result_output + '_evaluation.png', dpi=300)
    plt.clf()



    ###################### plot AUC


    plt.rcParams['figure.figsize'] = (15, 15)



    plt.xticks(fontsize = 15)
    plt.yticks(fontsize = 15)



    plt.plot(fpr*100, tpr*100, label='AUC = %.3f' % (auc))
    plt.plot(eff_99_FN*100, eff_99_ME*100,'*r', markersize=40)

    plt.plot(eff_95_FN*100, eff_95_ME*100,'*g', markersize=40)

    plt.plot(eff_90_FN*100, eff_90_ME*100,'*b', markersize=40)

    plt.plot(eff_50_FN*100, eff_50_ME*100,'*k', markersize=40)

    plt.xlabel('FN efficiency (%)', fontsize=15, loc='right')
    plt.ylabel('ME efficiency (%)', fontsize=15, loc='top')


    plt.xlim(0, 100)
    plt.ylim(0, 100)

    plt.grid()
    print_auc = '%.3f' %auc
    print_eff_99 = '%.3f' %eff_99
    print_eff_95 = '%.3f' %eff_95
    print_eff_90 = '%.3f' %eff_90
    label = ['AUC = '+print_auc,'99% WP(score='+print_eff_99+')', '95% WP(score='+print_eff_95+')', '90% WP(score='+print_eff_90+')','Middle WP (score=0.5)']

    leg = plt.legend(label, loc = 'right', frameon=False)


    leg_lines = leg.get_lines()
    leg_texts = leg.get_texts()

    plt.setp(leg_lines, linewidth=10)
    plt.setp(leg_texts, fontsize=10)

    plt.savefig('result/' + result_output + '/' + result_output + '_AUC.png', dpi=300)
    plt.clf()


    f = open('result/' + result_output + '/' + result_output + '_information.txt','w')


    print(eff_90_ME,eff_95_ME,eff_99_ME,eff_50_ME,file=f)

    print(eff_90_FN,eff_95_FN,eff_99_FN,eff_50_FN,file=f)

    print(int(num_90_ME),int(num_95_ME),int(num_99_ME),int(num_50_ME),file=f)

    print(int(num_90_FN),int(num_95_FN),int(num_99_FN),int(num_50_FN),file=f)
    
    print(eff_90_ME_FN,eff_95_ME_FN,eff_99_ME_FN,eff_50_ME,file=f)

    print(eff_90_FN_FN,eff_95_FN_FN,eff_99_FN_FN,eff_50_FN,file=f)

    print(int(num_90_ME_FN),int(num_95_ME_FN),int(num_99_ME_FN),int(num_50_ME),file=f)

    print(int(num_90_FN_FN),int(num_95_FN_FN),int(num_99_FN_FN),int(num_50_FN),file=f)

    print(eff_90,eff_95,eff_99,0.5,file=f)
    
    print(eff_90_range,eff_95_range,eff_99_range,0.5,file=f)


    print(len(FN_dT),len(ME_dT),file=f)
    print(auc,file=f)

    f.close()



In [ ]:
print(eff_90_range)
print(eff_95_range)
print(eff_99_range)